In [1]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = 'jhgan/ko-sbert-nli'
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
ko = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('/workspace/assets/doc/운전자보험상품약관.pdf')
pages = loader.load_and_split()

In [5]:
pages

[Document(page_content='보험약관 이 약관은 금융소비자보호법 및 소비자보호 내부통제기준에 따른 절차를 거쳐 제공됩니다.운전자보험\n2401.21무배당 삼성화재 다이렉트', metadata={'source': '/workspace/assets/doc/운전자보험상품약관.pdf', 'page': 0}),
 Document(page_content='2 / 278[ 목 차 ]\n약관이용 Guide Book 6\n쉽게 이해하는 상품 및 약관 요약서 9\n자주 발생하는 민원 예시 24\n보험용어 해설 27\n개인신용정보 제공·이용에 대한 고객 권리 안내문 28\n보통약관 29\n      제1관 목적 및 용어의 정의 31\n        제1조 (목적) 31\n        제2조 (용어의 정의) 31\n      제2관 보험금의 지급 31\n        제3조 (보험금의 지급사유) 31\n        제4조 (보험금 지급에 관한 세부규정) 32\n        제5조 (보험금을 지급하지 않는 사유) 33\n        제6조 (보험금 지급사유의 통지) 33\n        제7조 (보험금의 청구) 33\n        제8조 (보험금의 지급절차) 34\n        제9조 (공시이율의 적용 및 공시) 34\n        제10조 (만기환급금의 지급) 35\n        제11조 (보험금 받는 방법의 변경) 35\n        제12조 (주소변경통지) 35\n        제13조 (보험수익자의 지정) 35\n        제14조 (대표자의 지정) 35\n      제3관 계약자의 계약 전 알릴 의무 등 36\n        제15조 (계약 전 알릴 의무) 36\n        제16조 (상해보험계약 후 알릴 의무) 36\n        제17조 (알릴 의무 위반의 효과) 37        제18조 (사기에 의한 계약) 38\n      제4관 보험계약의 성립과 유지 38\n        제19조 (보험계약의 성립) 38\n        제

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=5, length_function=tiktoken_len)
docs = text_splitter.split_documents(pages)

In [7]:
from langchain.vectorstores import Chroma

# save to disk
saved_db = Chroma.from_documents(docs, ko, persist_directory='/workspace/chroma_db')
loaded_db = Chroma(persist_directory="/workspace/chroma_db", embedding_function=ko)


In [ ]:
query = "피보험자가 보험기간 중에 사망하였을 경우에는 보험료 및 해약환급금 산출방법서에서 정하는 바에 따라 회사가 적립한 사망당시 이 특별약관의 계약자적립액 및 미경과보험료를 지급하고, 이 특별약관은 더 이상 효력이 없습니다."
contents = loaded_db.similarity_search_with_relevance_scores(query)

In [ ]:
print("질문: {}\n".format(query))
for i in range(len(contents)) :
    print("{0}번째 유사 문서 유사도 \n{1}".format(i+1, round(contents[i][1],3)))
    print('-'*50)
    print(contents[i][0].page_content)
    print()
    print(contents[i][0].metadata)
    print('='*50)
    print()

질문: 달이 흐른다 5부 - 1고
(')

1번째 유사 문서 유사도 
0.225
--------------------------------------------------
10. 12월 25일 (기독탄신일 )

{'page': 270, 'source': '/workspace/assets/doc/운전자보험상품약관.pdf'}

2번째 유사 문서 유사도 
0.225
--------------------------------------------------
10. 12월 25일 (기독탄신일 )

{'page': 270, 'source': '/workspace/assets/doc/운전자보험상품약관.pdf'}

3번째 유사 문서 유사도 
0.223
--------------------------------------------------
중 먼저 도래하는  기간을 말합니다 .

{'page': 39, 'source': '/workspace/assets/doc/운전자보험상품약관.pdf'}

4번째 유사 문서 유사도 
0.223
--------------------------------------------------
중 먼저 도래하는  기간을 말합니다 .

{'page': 39, 'source': '/workspace/assets/doc/운전자보험상품약관.pdf'}



In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# from langchain_community.llms import LlamaCpp
from langchain.llms import LlamaCpp

model_path = "/workspace/models/llama-2-7b-chat.Q2_K.gguf"

llama = LlamaCpp(
    model_path=model_path,
    streaming=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /workspace/models/llama-2-7b-chat.Q2_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llama,
                                 chain_type="stuff",
                                 retriever=loaded_db.as_retriever(
                                     search_type="mmr",
                                     search_kwargs={'k':3, 'fetch_k': 10}
                                 ),
                                 return_source_documents=True
                                 )

In [12]:
query = "소비자가  반드시  알아두어야  할 상품의  주요 특성은 뭐야?"
result = qa(query)

/usr/local/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(

llama_print_timings:        load time =   38056.70 ms
llama_print_timings:      sample time =     341.52 ms /   256 runs   (    1.33 ms per token,   749.59 tokens per second)
llama_print_timings: prompt eval time = 1367780.66 ms /   238 tokens ( 5746.98 ms per token,     0.17 tokens per second)
llama_print_timings:        eval time = 10939860.17 ms /   255 runs   (42901.41 ms per token,     0.02 tokens per second)
llama_print_timings:       total time = 12344622.57 ms /   493 tokens


In [13]:
result

{'query': '소비자가  반드시  알아두어야  할 상품의  주요 특성은 뭐야?',
 'result': ' 소비자가 반드시 알아두어야 할 상품의 주요 특성은 다음과 같습니다.\n1. 확률: 소비자가 금융상품의 확률을 이해하면 됩니다. 이를 통해 소비자는 금융상품의  Risks 를 이해할 수 있습니다.\nExample: 독립 실수, 직접 손상, 직접 손상 등의 확률을 이해할 수 있습니다.\n2. 면직 기간: 소비자가 금융상품의 ',
 'source_documents': [Document(page_content='소비자가  반드시  알아두어야  할 상품의  주요 특성', metadata={'page': 14, 'source': '/workspace/assets/doc/운전자보험상품약관.pdf'}),
  Document(page_content='상품명으로  상품의  특징 이해하기', metadata={'page': 9, 'source': '/workspace/assets/doc/운전자보험상품약관.pdf'}),
  Document(page_content='는 다음 각 호의 금융상품에  관한 중요한  사항(일반금융소비자가  특정 사항에', metadata={'page': 272, 'source': '/workspace/assets/doc/운전자보험상품약관.pdf'})]}